In [1]:
import pandas as pd
from Utils import Utils

In [2]:
data_cleaner = Utils('assets/auto_train.csv')
data_cleaner.show_head()
df = data_cleaner.df

C:\Users\Luca Petrescu\Desktop\Master\Anul 1\Semestrul 2\DMDA\Curs\Tema 2\Utils.py:13: DtypeWarning: Columns (41,56) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_csv(self.file_path)


In [3]:
data_cleaner.get_nan_values()

id                                         0
data                                       0
nume                                       0
pret                                       0
url                                        0
Oferit de                                  0
Are VIN (Serie sasiu)                   8201
Marca                                      0
Model                                      0
Versiune                                4110
Anul fabricației                           0
Km                                        55
Combustibil                                0
Putere                                     7
Capacitate cilindrica                    554
Transmisie                              2184
Consum Extraurban                      11012
Cutie de viteze                            1
Consum Urban                            6150
Tip Caroserie                              0
Emisii CO2                              9621
Numar de portiere                        181
Culoare   

In [4]:
numeric_columns = data_cleaner.df.select_dtypes(include=['number'])
numeric_columns_with_nan = numeric_columns.columns[numeric_columns.isna().any()].tolist()
numeric_df_with_nan = df[numeric_columns_with_nan]
numeric_df_with_nan.info()
numeric_dataframe = pd.DataFrame(numeric_df_with_nan)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19526 entries, 0 to 19525
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Km                           19471 non-null  float64
 1   Putere                       19519 non-null  float64
 2   Capacitate cilindrica        18972 non-null  float64
 3   Consum Extraurban            8514 non-null   float64
 4   Consum Urban                 13376 non-null  float64
 5   Emisii CO2                   9905 non-null   float64
 6   Numar de portiere            19345 non-null  float64
 7   Numar locuri                 15264 non-null  float64
 8   sau in limita a              2926 non-null   float64
 9   Numar de rate lunare ramase  50 non-null     float64
dtypes: float64(10)
memory usage: 1.5 MB


In [5]:
object_columns = data_cleaner.df.select_dtypes(include=['object'])
object_columns_with_nan = object_columns.columns[object_columns.isna().any()].tolist()
object_df_with_nan = df[object_columns_with_nan]
object_df_with_nan.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19526 entries, 0 to 19525
Data columns (total 37 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   Are VIN (Serie sasiu)                11325 non-null  object
 1   Versiune                             15416 non-null  object
 2   Transmisie                           17342 non-null  object
 3   Cutie de viteze                      19525 non-null  object
 4   Se emite factura                     9541 non-null   object
 5   Eligibil pentru finantare            9428 non-null   object
 6   Garantie dealer (inclusa in pret)    5447 non-null   object
 7   Primul proprietar (de nou)           8066 non-null   object
 8   Fara accident in istoric             12135 non-null  object
 9   Carte de service                     12272 non-null  object
 10  VIN (serie sasiu)                    11330 non-null  object
 11  Audio si tehnologie                  1898

In [6]:
numeric_outliers = data_cleaner.detect_outliers(numeric_dataframe)

for col, outliers in numeric_outliers.items():
    print(f"Outliers in column '{col}':")
    print(outliers)
    print()


Outliers in column 'Km':
983       588000.0
1530      602000.0
2546     2080288.0
2982      488894.0
3208      510000.0
3323      500000.0
4729      620000.0
4754      412000.0
4937      489094.0
5058      412817.0
5340      411000.0
5696      504000.0
5724      440000.0
5733      502883.0
5898      413769.0
5999      469000.0
6640      458000.0
7405      550000.0
7481      449884.0
8543      450000.0
8862      418180.0
8906      415000.0
8931      575000.0
9019      800000.0
9268      547000.0
9410     1200002.0
10431     410000.0
11958     556000.0
12147     432631.0
12180     575000.0
12666     432280.0
13828     488504.0
14526     441903.0
14986     539151.0
15230     450000.0
15616     412000.0
16596     428000.0
16696     455000.0
16842     560000.0
17629     435000.0
17910     440000.0
18079     450000.0
18087     559780.0
18263     450000.0
18369    1630000.0
18890     572598.0
19196     413000.0
Name: Km, dtype: float64

Outliers in column 'Putere':
28       340.0
39       351

In [7]:
data_cleaner.fill_nan_with_median()
object_df_with_nan.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19526 entries, 0 to 19525
Data columns (total 37 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   Are VIN (Serie sasiu)                11325 non-null  object
 1   Versiune                             15416 non-null  object
 2   Transmisie                           17342 non-null  object
 3   Cutie de viteze                      19525 non-null  object
 4   Se emite factura                     9541 non-null   object
 5   Eligibil pentru finantare            9428 non-null   object
 6   Garantie dealer (inclusa in pret)    5447 non-null   object
 7   Primul proprietar (de nou)           8066 non-null   object
 8   Fara accident in istoric             12135 non-null  object
 9   Carte de service                     12272 non-null  object
 10  VIN (serie sasiu)                    11330 non-null  object
 11  Audio si tehnologie                  1898

In [8]:
data_cleaner.show_correlation()

C:\ProgramData\anaconda3\Lib\site-packages\seaborn\matrix.py:260: FutureWarning: Format strings passed to MaskedConstant are ignored, but in future may error or produce different behavior
  annotation = ("{:" + self.fmt + "}").format(val)


In [9]:
print(data_cleaner.fill_nan_with_frequent())

For column 'Are VIN (Serie sasiu)', filled NaN with 'Da'
For column 'Versiune', filled NaN with 'xDrive30d'
For column 'Transmisie', filled NaN with 'Fata'
For column 'Cutie de viteze', filled NaN with 'Automata'
For column 'Se emite factura', filled NaN with 'Da'
For column 'Eligibil pentru finantare', filled NaN with 'Da'
For column 'Garantie dealer (inclusa in pret)', filled NaN with '12 luni'
For column 'Primul proprietar (de nou)', filled NaN with 'Da'
For column 'Fara accident in istoric', filled NaN with 'Da'
For column 'Carte de service', filled NaN with 'Da'
For column 'VIN (serie sasiu)', filled NaN with 'Vezi VIN-ul (seria de sasiu)'
For column 'Audio si tehnologie', filled NaN with '['Radio']'
For column 'Confort si echipamente optionale', filled NaN with '['Aer conditionat', 'Geamuri electrice fata', 'Geamuri electrice spate']'
For column 'Electronice si sisteme de asistenta', filled NaN with '['Servodirectie']'
For column 'Siguranta', filled NaN with '['ABS', 'ESP', 'EBD'

In [10]:
pd.set_option('display.max_columns', 3000)